## Summed OS volume compared between the genotypes

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.multicomp as mc

from scipy.stats import ttest_ind

In [2]:
path = "../../measurements/cpfl/"
measurements = pd.read_csv(path + "04-measurements-rescaled.csv")
measurements

,label,age,image_id,genotype,maximum,mean,median,minimum,sigma,sum,...,equivalent_spherical_perimeter_rescaled,equivalent_spherical_radius_rescaled,feret_diameter_rescaled,perimeter_2d_rescaled,major_axis_length_2d_rescaled,minor_axis_length_2d_rescaled,surface_area_rescaled,bbox_volume_rescaled,convex_volume_rescaled,volume_rescaled
0,1,8,0,cpfl,250.0,189.793103,184.144531,157.0,20.305487,5504.0,...,14.744113,0.615610,1.480172,2.957153,0.976660,0.904212,3.881768,1.516422,1.078345,0.977250
1,2,8,0,cpfl,334.0,207.273684,202.871094,133.0,42.706679,19691.0,...,32.521386,0.914283,2.624064,4.973873,2.127172,1.027014,7.095704,4.852550,3.572016,3.201335
2,4,8,0,cpfl,262.0,214.904762,215.355469,175.0,27.285353,4513.0,...,11.889571,0.552814,1.370373,2.957153,0.976660,0.904212,5.174290,1.516422,0.943551,0.707664
3,5,8,0,cpfl,311.0,195.171429,190.386719,118.0,36.997206,6831.0,...,16.713421,0.655434,1.678357,3.390218,1.255098,0.902928,7.379101,2.426275,1.381629,1.179439
4,6,8,0,cpfl,355.0,219.354167,215.355469,147.0,47.335750,21058.0,...,32.749208,0.917480,2.068209,5.481241,1.910113,1.514661,11.022045,6.065688,3.774206,3.235034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5845,176,70,39,wt,437.0,228.476298,219.855469,118.0,59.914239,101215.0,...,90.772814,1.527475,5.732681,13.307188,6.005873,1.731845,45.719796,55.197761,20.994020,14.928332
5846,177,70,39,wt,1542.0,485.858939,412.605469,137.0,265.029227,1043625.0,...,260.041601,2.585340,9.158631,32.454379,8.011704,7.033195,160.768595,300.992921,132.366793,72.383878
5847,178,70,39,wt,416.0,224.548673,213.832031,127.0,62.839505,25374.0,...,36.509477,0.968722,2.261000,5.734924,1.901773,1.715924,15.687029,8.491963,4.414473,3.807904
5848,180,70,39,wt,481.0,251.469466,237.925781,145.0,64.019970,65885.0,...,63.956915,1.282154,4.579315,13.771030,4.629315,2.203200,35.317111,35.383180,15.063125,8.828946


In [3]:
volume_sum = measurements.groupby("image_id", as_index=False)["volume_rescaled"].sum()
age = measurements.groupby("image_id", as_index=False)["age", "genotype"].max()
volume_sum = pd.merge(volume_sum,
                      age,
                      how="inner")
volume_sum

C:\Users\fsalo\AppData\Local\Temp\ipykernel_6244\1930686293.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  age = measurements.groupby("image_id", as_index=False)["age", "genotype"].max()


,image_id,volume_rescaled,age,genotype
0,0,563.030645,8,cpfl
1,1,648.556847,8,cpfl
2,2,300.083068,8,cpfl
3,3,3659.193719,14,cpfl
4,4,3710.954257,14,cpfl
5,5,2371.852523,14,cpfl
6,6,2115.038030,20,cpfl
7,7,1804.643293,20,cpfl
8,8,2869.171547,20,cpfl
9,9,953.964241,245,cpfl


In [6]:
ages = [8, 14, 20, 30, 70, 245]

for i, postnatal_age in enumerate(ages):
    
    age_filtered = volume_sum[volume_sum["age"] == postnatal_age]
    
    volumes_cpfl = age_filtered[age_filtered["genotype"] == "cpfl"]["volume_rescaled"]
    volumes_wt = age_filtered[age_filtered["genotype"] == "wt"]["volume_rescaled"]
    
    t_statistic, p_value = ttest_ind(volumes_wt,
                                     volumes_cpfl,
                                     equal_var=False,
                                     alternative="greater")
    
    print("Statistic at postnatal age ", postnatal_age)
    print("T-statistics :", np.round(t_statistic, 3))
    print("P-value :", np.round(p_value, 3))

Statistic at postnatal age  8
T-statistics : 3.038
P-value : 0.016
Statistic at postnatal age  14
T-statistics : -0.97
P-value : 0.81
Statistic at postnatal age  20
T-statistics : -0.259
P-value : 0.596
Statistic at postnatal age  30
T-statistics : 3.333
P-value : 0.015
Statistic at postnatal age  70
T-statistics : 3.177
P-value : 0.04
Statistic at postnatal age  245
T-statistics : 9.375
P-value : 0.002
